<a href="https://colab.research.google.com/github/pavankumarallu/Bolt-PRO/blob/main/BOLTPRO_SEQGEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split 
from keras.initializers import RandomNormal
from keras.layers import Dense, Input, Reshape, Dropout, ZeroPadding2D
from keras.layers import ELU, PReLU, LeakyReLU
from keras.models import Model, Sequential
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from keras.models import load_model

In [31]:

df2 = pd.read_csv('/content/drive/MyDrive/Sequence_HSepians.csv')
# np.random.seed(1337)
noise_dim = 50
batch_size = 64
output_dim=500
epochs = 100
optimizer = Adam(0.0002, 0.5)
df2

,simple_fasta
0,MSLIQKEAQGQSGTDQTVVVLSNPTYYMSNDIPYTFHQDNNFLYLC...
1,MQRDHTMDYKESCPSVSIPSSDEHREKKKRFTVYKVLVSVGRSEWF...
2,MARLSGTVGVAAVTAGPGLTNTVTAVKNAQMAQSPILLLGGAASTL...
3,MATSLDFKTYVDQACRAAEEFVNIYYETMDKRRRALTRLYLDKATL...
4,MVFRRFVEVGRVAYVSFGPHAGKLVAIVDVIDQNRALVDGPCTQVR...
...,...
6451,MADVLDLHEAGGEDFAMDEDGDESIHKLKEKAKKRKGRGFGSEEGS...
6452,MDAPRQVVNFGPGPAKLPHSVLLEIQKELLDYKGVGISVLEMSHRS...
6453,MPKTMHFLFRFIVFFYLWGLFTAQRQKKEESTEEVKIEVLHRPENC...
6454,MSLQVLNDKNVSNEKNTENCDFLFSPPEVTGRSSVLRVSQKENVPP...


In [29]:
train_sequence = df2['simple_fasta']

tokenizer = Tokenizer(lower=False, char_level=True)
tokenizer.fit_on_texts(train_sequence)

train_sequence = tokenizer.texts_to_sequences(train_sequence)
train_sequence = pad_sequences(train_sequence, maxlen=500)

In [30]:
train_sequence

array([[ 0,  0,  0, ..., 12,  4,  3],
       [ 0,  0,  0, ...,  3, 10,  8],
       [ 0,  0,  0, ...,  8, 12, 12],
       ...,
       [ 0,  0,  0, ..., 10,  2,  1],
       [ 2, 15, 10, ...,  2,  5, 13],
       [ 0,  0,  0, ..., 13, 16, 14]], dtype=int32)

In [32]:
def create_generator(noise_dim, out_dim):
    generator = Sequential()

    generator.add(Dense(256,input_dim = noise_dim))
    generator.add(LeakyReLU(0.2));

    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2));

    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(out_dim))
    generator.add(LeakyReLU(0.2))

    generator.compile(loss='binary_crossentropy',
                      optimizer = optimizer)

    return generator

  

In [33]:
def create_descriminator(input_dimension):
    discriminator = Sequential()
     
    discriminator.add(Dense(1024, input_dim=input_dimension))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', 
                          optimizer=optimizer, 
                          metrics=['accuracy'])
    return discriminator
     

In [34]:


discriminator = create_descriminator(output_dim)
generator = create_generator(noise_dim, output_dim)

discriminator.trainable = False

gan_input = Input(shape=(noise_dim,))
fake_sequence = generator(gan_input)

gan_output = discriminator(fake_sequence)

gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [35]:

# Display images, and save them if the epoch number is specified
def show_images():
    noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
    generated_images = generator.predict(noise)
    
    
    for i, image in enumerate(generated_images):
        image = tokenizer.sequences_to_texts(train_sequence)
        print(image)


In [37]:

for epoch in range(100):
    for batch in range(20):
        for _ in range(6):

            noise = np.random.normal(0, 1, size=(batch_size, noise_dim))

            real_x = train_sequence[np.random.randint(0, train_sequence.shape[0], size=batch_size)]
            fake_x = generator.predict(noise)

            x = np.concatenate((real_x, fake_x))

            disc_y = np.zeros(2*batch_size)
            disc_y[:batch_size] = 0.95



            discriminator.train_on_batch(x, disc_y)
        for _ in range(2):
            noise = np.random.normal(0, 1, size=(batch_size, noise_dim))

            y_gen = np.ones(batch_size)
            gan.train_on_batch(noise, y_gen)
    print(f'Epoch: {epoch}')
    # print(f'Epoch: {epoch} \t Discriminator Loss: {d_loss} \t\t Generator Loss: {g_loss}')
    #if epoch % 10 == 0:
        #show_images()
generator.save('/content/drive/My Drive/BoltPro/generator.h5')
gan.save('/content/drive/My Drive/BoltPro/gan.h5')
discriminator.save('/content/drive/My Drive/BoltPro/discriminator.h5')
     

Streaming output truncated to the last 5000 lines.
2/2 [==============================] - 0s 9ms/step
Epoch: 58
2/2 [==============================] - 0s 9ms/step
Epoch: 59
2/2 [==============================] - 0s 11ms/step
Epoch: 60
2/2 [==============================] - 0s 11ms/step
Epoch: 61
2/2 [==============================] - 0s 15ms/step
Epoch: 62
2/2 [==============================] - 0s 11ms/step
Epoch: 63
2/2 [==============================] - 0s 15ms/step
Epoch: 64
2/2 [==============================] - 0s 10ms/step
Epoch: 65
2/2 [==============================] - 0s 17ms/step
Epoch: 66
2/2 [==============================] - 0s 9ms/step
Epoch: 67
2/2 [==============================] - 0s 21ms/step
Epoch: 68
2/2 [==============================] - 0s 9ms/step
Epoch: 69
2/2 [==============================] - 0s 13ms/step
Epoch: 70
2/2 [==============================] - 0s 9ms/step
Epoch: 71
2/2 [==============================] - 0s 13ms/step
Epoch: 72
2/2 [===================

In [38]:
loaded_model = load_model('/content/drive/My Drive/BoltPro/generator.h5')

In [53]:
noise = np.random.normal(0, 1, size=(1, noise_dim))
#generated_sequence = generator.predict(noise)
generated_sequence = loaded_model.predict(noise)
generated_sequence = generated_sequence.astype(int)


1/1 [==============================] - 0s 18ms/step


In [54]:
a = train_sequence.shape[1]
generated_text = tokenizer.sequences_to_texts([generated_sequence[0][:a]])
s = generated_text[0].replace(" ", "")

In [55]:
s

'LLLKESGSILLSPALKRSGVKVKTVSSDQSEVGVRGKPKPQSDSARGEAPKPRTQRKKGVPIPLEYPPTPAAGALVLGSVAGVAVRVKNANKPRDQVKGPTPSPQSEKEVRAKKKTTPRGKDAAKPIVLKVPVARVINIPKPSARDNDITERAAPPFAIVAPDKAPGVRGFRRADKGGEVDPSKDEQPSPEDPDVREDVSKTIGVDKLRATKEVDADGAERRPGDRVDDGRTRASGVGFVRPGPDEKLSKSDKRRDAPPVAETPDRAGEAPTKKDPRGRDLQSGLPGDAQKRNKSPKVGAEKGVVEGRNENTDTDVTPGGADTLNRAEESRGRIVSGGYGTNGGIGTSTRTTDVPPRATPATSKSRTKFKYVSPITVPVAPKVSKPALIAPRKGTTVDIVSSPDGGSSPKKVLRGVSTGSSGGIRRKAPRAKVVDDP'